# This is a simple jupyte notebook to test parallel programming in Python

## first install the followings:


### pip install conda
### conda install -c conda-forge numpy scipy joblib jupyter time

In [1]:

import numpy as np
import time
from scipy import ndimage
from joblib import Parallel, delayed

def array_for_sliding_window(x, wshape):
 
    x = np.asarray(x)

    try:
        nd = len(wshape)
    except TypeError:
        wshape = tuple(wshape for i in x.shape)
        nd = len(wshape)
    if nd != x.ndim:
        raise ValueError("wshape has length {0} instead of "
                         "x.ndim which is {1}".format(len(wshape), x.ndim)) 
    
    out_shape = tuple(xi-wi+1 for xi, wi in zip(x.shape, wshape)) + wshape
    if not all(i>0 for i in out_shape):
        raise ValueError("wshape is bigger than input array along at "
                         "least one dimension")

    out_strides = x.strides*2
    
    return np.lib.stride_tricks.as_strided(x, out_shape, out_strides)


def marfurt_semblance2(region):
    region = region.reshape(-1, region.shape[-1])
    ntraces, nsamples = region.shape

    cov = region.dot(region.T)
    sembl = cov.sum() / cov.diagonal().sum()
    return sembl / ntraces


In [2]:
np.random.seed(10)
a=np.random.random((100,100,100))
window=(2, 2,2)


print('running in a single core')

s=time.time()
wrapped = lambda region: marfurt_semblance2(region.reshape(window))
r0=ndimage.generic_filter(a, wrapped, window)
print('Elapsed time(seconds): ',round(time.time()-s,2))



running in a single core
Elapsed time(seconds):  9.05


In [3]:


b=array_for_sliding_window(a,window)
print(b.shape)
r=np.zeros(a.shape)




def runit(nrun=60):    
    print('running in parallel')
    a=Parallel(n_jobs=nrun,prefer="threads")(
            delayed(marfurt_semblance2)(b[i]) for i in np.ndindex(b.shape[:-len(window)]))
    return(a)

if __name__ == '__main__':
    s=time.time()
    a=runit(nrun=60)
    #a=np.array(a,)
    print('Elapsed time: ',round(time.time()-s,2))



(99, 99, 99, 2, 2, 2)
running in parallel
Elapsed time:  245.61
